# importação de Bibliotecas necessárias

In [2]:
!pip install yt-dlp openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.0 MB/s eta 0:00:00


In [9]:
import yt_dlp
from openai import OpenAI
import os
import getpass

# Baixando e convertendo vídeo em áudio

In [4]:
url = input("Qual a URL do seu vídeo?")

Qual a URL do seu vídeo?https://www.youtube.com/watch?v=5oz5dwHU_mM


In [14]:
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '64',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

print("Download concluído!")

[youtube] Extracting URL: https://www.youtube.com/watch?v=5oz5dwHU_mM
[youtube] 5oz5dwHU_mM: Downloading webpage
[youtube] 5oz5dwHU_mM: Downloading tv client config
[youtube] 5oz5dwHU_mM: Downloading player d50f54ef
[youtube] 5oz5dwHU_mM: Downloading tv player API JSON
[youtube] 5oz5dwHU_mM: Downloading ios player API JSON
[youtube] 5oz5dwHU_mM: Downloading m3u8 information
[info] 5oz5dwHU_mM: Downloading 1 format(s): 251
[download] Destination: Intro to Databricks Lakehouse Platform Architecture and Security [5oz5dwHU_mM].webm
[download] 100% of   22.76MiB in 00:00:01 at 15.77MiB/s  
[ExtractAudio] Destination: Intro to Databricks Lakehouse Platform Architecture and Security [5oz5dwHU_mM].mp3
Deleting original file Intro to Databricks Lakehouse Platform Architecture and Security [5oz5dwHU_mM].webm (pass -k to keep)
Download concluído!


# Transformando o Áudio em Texto

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI Api Key: ")

OpenAI Api Key: ··········


In [11]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

Observação importante: O arquivo deve ter no máximo 25MB (por isso baixei o áudio na pior qualidade possível).

Mas, você pode picotar o arquivo original em partes menores, baixar cada parte, resumir essas partes e juntar no final. Maior rolê, mas que dá pra fazer, dá! xD

---
Lembre-se de atualizar o caminho para o local onde o seu arquivo de áudio realmente está, ok?

In [15]:
audio_file= open("/content/Intro to Databricks Lakehouse Platform Architecture and Security [5oz5dwHU_mM].mp3", "rb")
transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

In [16]:
print(transcription.text)

Databricks Lakehouse Platform Architecture and Security Fundamentals Data Reliability and Performance. In this video, you'll learn about the importance of data reliability and performance on platform architecture, define Delta Lake, and describe how Photon improves the performance of the Databricks Lakehouse platform. First, we'll address why data reliability and performance is important. It is common knowledge that bad data in equals bad data out, so the data used to build business insights and draw actionable conclusions needs to be reliable and clean. While data lakes are a great solution for holding large quantities of raw data, they lack important features for data reliability and quality, often leading them to be called data swamps. Also, data lakes don't often offer as good of performance as that of data warehouses. Some of the problems data engineers may encounter when using a standard data lake include a lack of ACID transaction support, making it impossible to mix updates, ap

# Resumindo o texto!

In [26]:
prompt = f"""
  Você é um redator excepcional, com um olhar clínico para detalhes e uma habilidade única para captar todos os pontos importantes.
  Sua tarefa é transformar o conteúdo fornecido em um resumo claro, conciso e organizado em Markdown, perfeito para anotações.
  Lembre-se: cada detalhe é crucial, pois o resumo será utilizado para referência posterior.
  Você pode iniciar o resumo diretamente, sem a necessidade de introduções como "o vídeo trata de...".
  Foque diretamente nos elementos essenciais e relevantes do texto.

  Abaixo, segue o texto a ser resumido:
  {transcription.text}
 """

In [27]:
llm_model = "gpt-4o-mini"

In [28]:
def get_completion(prompt, model=llm_model):
    messages = [{"role":"user", "content":prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [29]:
resposta = get_completion(prompt)
resposta

'# Resumo: Databricks Lakehouse Platform Architecture and Security Fundamentals\n\n## Importância da Confiabilidade e Desempenho dos Dados\n- **Confiabilidade dos Dados**: Dados ruins resultam em insights ruins. É crucial que os dados sejam confiáveis e limpos.\n- **Desempenho**: Data lakes, embora armazenem grandes quantidades de dados, carecem de recursos para garantir a qualidade e confiabilidade, frequentemente sendo chamados de "data swamps".\n- **Problemas com Data Lakes**:\n  - Falta de suporte a transações ACID.\n  - Ausência de imposição de esquema, resultando em dados inconsistentes.\n  - Integração deficiente com catálogos de dados, levando a "dark data".\n\n## Soluções da Databricks\n### Delta Lake\n- **Formato de Armazenamento**: Open-source, baseado em arquivos.\n- **Garantias**:\n  - Suporte a transações ACID.\n  - Manipulação escalável de dados e metadados.\n  - Histórico de auditoria e "time travel".\n  - Evolução de esquema e imposição de esquema.\n  - Suporte para op

GG, agora é só colar o resumo no seu Notion ^^ !